In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [2]:
train = []
label_train = []
test = []
label_test = []

In [3]:
def load_images_from_path(path, label):
    images = []
    labels = []

    for file in os.listdir(path):
        try:
            image_path = os.path.join(path, file)
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(label)
        except Exception as e:
            print(f"Error loading image {file} with OpenCV: {str(e)}")

    return images, labels

In [4]:
images, labels = load_images_from_path('/kaggle/input/chest-x-ray-image/Data/train/COVID19', 0)
    
train += images
label_train += labels

images, labels = load_images_from_path('/kaggle/input/chest-x-ray-image/Data/train/NORMAL', 1)
    
train += images
label_train += labels

images, labels = load_images_from_path('/kaggle/input/chest-x-ray-image/Data/train/PNEUMONIA', 2)
    
train += images
label_train += labels

images, labels = load_images_from_path('/kaggle/input/chest-x-ray-image/Data/test/COVID19', 0)
    
test += images
label_test += labels

images, labels = load_images_from_path('/kaggle/input/chest-x-ray-image/Data/test/NORMAL', 1)
    
test += images
label_test += labels

images, labels = load_images_from_path('/kaggle/input/chest-x-ray-image/Data/test/PNEUMONIA', 2)
    
test += images
label_test += labels

In [5]:
train_norm= np.array(train) / 255
test_norm= np.array(test) / 255
val_norm= np.array(val) / 255

label_train_encoded = to_categorical(label_train)
label_test_encoded = to_categorical(label_test)
label_val_encoded = to_categorical(label_val)

In [ ]:
split_index = len(test) // 2

val = test[:split_index]
label_val = label_test[:split_index]

modified_test = test[split_index:]
label_modified_test = label_test[split_index:]

test = modified_test
label_test = label_modified_test

In [7]:
positional_emb = True
conv_layers = 2
projection_dim = 128

num_heads = 2
transformer_units = [
    projection_dim,
    projection_dim,
]
transformer_layers = 2
stochastic_depth_rate = 0.1

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 128
num_epochs = 300
image_size = 224

num_classes = 3
input_shape = (224, 224, 3)

In [8]:
class CCTTokenizer(layers.Layer):
    def __init__(
        self,
        kernel_size=3,
        stride=1,
        padding=1,
        pooling_kernel_size=3,
        pooling_stride=2,
        num_conv_layers=conv_layers,
        num_output_channels=[64, 128],
        positional_emb=positional_emb,
        **kwargs,
    ):
        super().__init__(**kwargs)

        # This is our tokenizer.
        self.conv_model = keras.Sequential()
        for i in range(num_conv_layers):
            self.conv_model.add(
                layers.Conv2D(
                    num_output_channels[i],
                    kernel_size,
                    stride,
                    padding="valid",
                    use_bias=False,
                    activation="relu",
                    kernel_initializer="he_normal",
                )
            )
            self.conv_model.add(layers.ZeroPadding2D(padding))
            self.conv_model.add(
                layers.MaxPool2D(pooling_kernel_size, pooling_stride, "same")
            )

        self.positional_emb = positional_emb

    def call(self, images):
        outputs = self.conv_model(images)
        # After passing the images through our mini-network the spatial dimensions
        # are flattened to form sequences.
        reshaped = tf.reshape(
            outputs,
            (-1, tf.shape(outputs)[1] * tf.shape(outputs)[2], tf.shape(outputs)[-1]),
        )
        return reshaped

    def positional_embedding(self, image_size):
        # Positional embeddings are optional in CCT. Here, we calculate
        # the number of sequences and initialize an `Embedding` layer to
        # compute the positional embeddings later.
        if self.positional_emb:
            dummy_inputs = tf.ones((1, image_size, image_size, 3))
            dummy_outputs = self.call(dummy_inputs)
            sequence_length = tf.shape(dummy_outputs)[1]
            projection_dim = tf.shape(dummy_outputs)[-1]

            embed_layer = layers.Embedding(
                input_dim=sequence_length, output_dim=projection_dim
            )
            return embed_layer, sequence_length
        else:
            return None

In [9]:

class StochasticDepth(layers.Layer):
    def __init__(self, drop_prop, **kwargs):
        super().__init__(**kwargs)
        self.drop_prob = drop_prop

    def call(self, x, training=None):
        if training:
            keep_prob = 1 - self.drop_prob
            shape = (tf.shape(x)[0],) + (1,) * (tf.shape(x).shape[0] - 1)
            random_tensor = keep_prob + tf.random.uniform(shape, 0, 1)
            random_tensor = tf.floor(random_tensor)
            return (x / keep_prob) * random_tensor
        return x

In [10]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomCrop(image_size, image_size),
        layers.RandomFlip("horizontal"),
    ],
    name="data_augmentation",
)

In [12]:
def create_cct_model(
    image_size=image_size,
    input_shape=input_shape,
    num_heads=num_heads,
    projection_dim=projection_dim,
    transformer_units=transformer_units,
):
    inputs = layers.Input(input_shape)

    # Augment data.
    augmented = data_augmentation(inputs)

    # Encode patches.
    cct_tokenizer = CCTTokenizer()
    encoded_patches = cct_tokenizer(augmented)

    # Apply positional embedding.
    if positional_emb:
        pos_embed, seq_length = cct_tokenizer.positional_embedding(image_size)
        positions = tf.range(start=0, limit=seq_length, delta=1)
        position_embeddings = pos_embed(positions)
        encoded_patches += position_embeddings

    # Calculate Stochastic Depth probabilities.
    dpr = [x for x in np.linspace(0, stochastic_depth_rate, transformer_layers)]

    # Create multiple layers of the Transformer block.
    for i in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)

        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)

        # Skip connection 1.
        attention_output = StochasticDepth(dpr[i])(attention_output)
        x2 = layers.Add()([attention_output, encoded_patches])

        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-5)(x2)

        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)

        # Skip connection 2.
        x3 = StochasticDepth(dpr[i])(x3)
        encoded_patches = layers.Add()([x3, x2])

    # Apply sequence pooling.
    representation = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)
    attention_weights = tf.nn.softmax(layers.Dense(1)(representation), axis=1)
    weighted_representation = tf.matmul(
        attention_weights, representation, transpose_a=True
    )
    weighted_representation = tf.squeeze(weighted_representation, -2)

    # Classify outputs.
    logits = layers.Dense(num_classes)(weighted_representation)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [13]:
# Detect TPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

# Select appropriate distribution strategy
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


REPLICAS:  8


In [14]:
# Create and compile the model inside the TPU scope
with strategy.scope():
    model = create_cct_model()
    model.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
        optimizer=keras.optimizers.AdamW(
            learning_rate=0.001, weight_decay=0.0001
        ),
        metrics=[
            keras.metrics.CategoricalAccuracy(name="accuracy"),
        ],
    )

# Define EarlyStopping callback
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=15,
    restore_best_weights=True
)

# Train the model with EarlyStopping
history = model.fit(
    train_norm,
    label_train_encoded,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_data=(val_norm, label_val_encoded),
    callbacks=[early_stopping]
)

# After training, calculate and print metrics
predictions = model.predict(test_norm)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(label_test_encoded, axis=1)

accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
f1 = f1_score(true_labels, predicted_labels, average='weighted')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')



2023-12-04 16:14:50.650737: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-04 16:14:50.650828: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


Epoch 1/300


2023-12-04 16:15:02.230156: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-04 16:15:02.247512: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-04 16:15:02.264864: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-04 16:15:02.281946: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-04 16:15:02.300493: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-04 16:15:02.319221: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-04 16:15:02.337016: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

 1/41 [..............................] - ETA: 11:19 - loss: 2.0086 - accuracy: 0.0703

2023-12-04 16:15:19.207813: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-04 16:15:19.208243: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-04 16:15:19.208343: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-04 16:15:19.208429: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-04 16:15:19.208526: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-04 16:15:19.208637: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-04 16:15:19.208741: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

...

41/41 [==============================] - 7s 174ms/step - loss: 0.7029 - accuracy: 0.7319 - val_loss: 0.6779 - val_accuracy: 0.7492
Epoch 3/300
41/41 [==============================] - 7s 173ms/step - loss: 0.6289 - accuracy: 0.7830 - val_loss: 0.6323 - val_accuracy: 0.7826
Epoch 4/300
41/41 [==============================] - ETA: 0s - loss: 0.5732 - accuracy: 0.8293

2023-12-04 16:16:08.605461: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


41/41 [==============================] - 7s 173ms/step - loss: 0.5732 - accuracy: 0.8293 - val_loss: 0.5894 - val_accuracy: 0.8370
Epoch 5/300
41/41 [==============================] - 7s 171ms/step - loss: 0.5392 - accuracy: 0.8546 - val_loss: 0.5780 - val_accuracy: 0.8377
Epoch 6/300
41/41 [==============================] - 7s 172ms/step - loss: 0.5090 - accuracy: 0.8746 - val_loss: 0.5250 - val_accuracy: 0.8750
Epoch 7/300
41/41 [==============================] - 7s 173ms/step - loss: 0.5106 - accuracy: 0.8748 - val_loss: 0.5208 - val_accuracy: 0.8859
Epoch 8/300
41/41 [==============================] - 7s 170ms/step - loss: 0.4880 - accuracy: 0.8853 - val_loss: 0.5148 - val_accuracy: 0.8797
Epoch 9/300
41/41 [==============================] - 7s 171ms/step - loss: 0.4893 - accuracy: 0.8867 - val_loss: 0.5080 - val_accuracy: 0.8859
Epoch 10/300
41/41 [==============================] - 7s 171ms/step - loss: 0.4605 - accuracy: 0.9092 - val_loss: 0.5833 - val_accuracy: 0.8370
Epoch 11/3

2023-12-04 16:24:50.138571: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


41/41 [==============================] - 16s 139ms/step
Accuracy: 0.9379
Precision: 0.9374
Recall: 0.9379
F1 Score: 0.9375
